# Prepare Data for Predictive Maintenance  

### Please don't run / don't click "Run all" the notebook:
At the time of writing of this document, the current core limit is 200 cores per workspace and depending upon number of concurrent users, you may end up with core capacity being exceeded or maximum number of parallel jobs being exceeded error.   


## Overview
*Dataset*: Telemetry data from 1000 machines **(8.6 Million events)** with reference data for machine failures, errors, past routine maintenance, and general information about machines. Time series features were extracted from telemetry data.

*Tools/Techniques*: AutoML 

### Notebook Organization 
+ Data Ingestion from ADLSGen2 

+ Data preparation using PySpark in Synapse 

#### Data Ingestion


In [5]:
# Load the libraries
from pyspark.sql import SparkSession

# Create the pyspark environment
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [6]:
# Define urls for all data
DATA_BASE_URI='abfss://mfgdemodata@#DATA_LAKE_NAME#.dfs.core.windows.net/'
MACHINES_URI= DATA_BASE_URI + "machines.csv"
ERRORS_URI= DATA_BASE_URI + "errors.csv"
MAINT_URI= DATA_BASE_URI + "maint.csv"
FAILURES_URI = DATA_BASE_URI + "failures.csv"
TELEMETRY_URI= DATA_BASE_URI + "telemetryp.csv"

# Define hive table names for all data
MACHINES_TABLE = "machines_table"
ERRORS_TABLE = "errors_table"
MAINT_TABLE="maint_table"
FAILURES_TABLE="failures_table"
TELEMETRY_TABLE="telemetry_table"
FEATURES_TABLE="final_features_table"
TRAIN_TABLE="train_table"
TEST_TABLE = "test_table"
SCORE_TABLE = "score_table"
RESULTS_TABLE= "results_table"

# Dates for Sampling
TRAIN_START_DATE='2000-01-01'
TRAIN_END_DATE='2015-10-30'
TEST_START_DATE='2015-11-30' 
TEST_END_DATE = '2016-02-01' 
SCORE_START_DATE='2015-11-30'
SCORE_END_DATE= '2016-04-30'

# Model Version
MODEL_VERSION = "V1"

1. ##### Machine Dataset


In [5]:
# Download the data and save the dataframe into Hive table if not present
if not(spark.catalog._jcatalog.tableExists(MACHINES_TABLE)):
   machine_spark = spark.read.load(MACHINES_URI, format='csv',header=True,inferschema=True)
   machine_spark.write.saveAsTable(MACHINES_TABLE)

# Display machine dataset
display(spark.table(MACHINES_TABLE))

##### Errors Data set


In [8]:
# Download the data and save the dataframe into Hive table if not present
if not(spark.catalog._jcatalog.tableExists(ERRORS_TABLE)):
   error_spark = spark.read.load(ERRORS_URI, format='csv',header=True,inferschema=True)
   error_spark.write.saveAsTable(ERRORS_TABLE)

# Display machine dataset
display(spark.table(ERRORS_TABLE))

##### Maintenance data set

In [9]:
# Download the data and save the dataframe into Hive table if not present
if not(spark.catalog._jcatalog.tableExists(MAINT_TABLE)):
   maint_spark = spark.read.load(MAINT_URI, format='csv',header=True,inferschema=True)
   maint_spark.write.saveAsTable(MAINT_TABLE)

# Display machine dataset
display(spark.table(MAINT_TABLE))

##### Telemetry Dataset


In [10]:
# Download the data and save the dataframe into Hive table if not present
if not(spark.catalog._jcatalog.tableExists(TELEMETRY_TABLE)):
   telemetry_spark = spark.read.load(TELEMETRY_URI, format='csv',header=True,inferschema=True)
   telemetry_spark.write.saveAsTable(TELEMETRY_TABLE)

# Display machine dataset
display(spark.table(TELEMETRY_TABLE))

##### Failures data set


In [11]:
# Download the data and save the dataframe into Hive table if not present
if not(spark.catalog._jcatalog.tableExists(FAILURES_TABLE)):
   failures_spark = spark.read.load(FAILURES_URI, format='csv',header=True,inferschema=True)
   failures_spark.write.saveAsTable(FAILURES_TABLE)

# Display machine dataset
display(spark.table(FAILURES_TABLE))

#### Exploratory Data Analysis


In [12]:
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import pandas as pd

In [13]:
## Getting pandas datframes for data visualization
machines_pd = spark.table(MACHINES_TABLE).toPandas()
errors_pd = spark.table(ERRORS_TABLE).toPandas()
maint_pd = spark.table(MAINT_TABLE).toPandas()
failures_pd  = spark.table(FAILURES_TABLE).toPandas() 
telemetry_pd = spark.table(TELEMETRY_TABLE).toPandas()

/home/trusted-service-user/cluster-env/env/lib/python3.6/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
  warnings.warn("pyarrow.open_stream is deprecated, please use "

In [14]:
# matplotlib styling 
plt.style.use('seaborn-whitegrid')
# contoliing plot size 
fig = plt.figure(figsize=(12,4))
# matplotlib histogram
plt.hist(machines_pd['model'], color = 'steelblue', edgecolor = 'steelblue') 
plt.xlabel('Models')
plt.ylabel('Number of Machines')
plt.show()

In [15]:
# matplotlib styling 
plt.style.use('seaborn-whitegrid')
# contoliing plot size 
fig = plt.figure(figsize=(12,6))

# stacked histograms of models by age 
plt.hist([machines_pd.loc[machines_pd['model'] == 'model1', 'age'],
         machines_pd.loc[machines_pd['model'] == 'model2', 'age'],
         machines_pd.loc[machines_pd['model'] == 'model3', 'age'],
         machines_pd.loc[machines_pd['model'] == 'model4', 'age']],
         20,
         stacked = True,
         color= ['skyblue', 'olive', 'bisque', 'lightseagreen'],
         edgecolor='dimgrey',
         label=['model 1', 'model 2', 'model 3', 'model 4'])
plt.legend() 
plt.xlabel('Service Age (yrs)')
plt.ylabel('Number of Machines')
plt.show()

In [16]:
# matplotlib styling 
plt.style.use('seaborn-whitegrid')
# contoliing plot size 
fig = plt.figure(figsize=(12,4))

# matplotlib histogram
plt.hist(maint_pd['comp'], color = 'darkgreen', edgecolor = 'darkgreen') 

plt.xlabel('Maintenance Types (by components)')
plt.ylabel('Number of Maitenance Done')
plt.show()

In [17]:
# format datetime field which comes in as string
maint_pd['datetime'] = pd.to_datetime(maint_pd['datetime'], format="%Y-%m-%d %H:%M:%S") 

# seaborn styling 
sns.set_style("whitegrid")

# contoliing plot size 
fig, axes = plt.subplots(1,1, figsize=(12,6))
# fig = plt.figure(figsize=(12,4))

# grouping failures data   
maint_pd_grp = maint_pd.groupby(['datetime', 'comp']).size().reset_index(name = 'maint_counts')

# plotting by diffferent failures
sns.lineplot(x = 'datetime', y = 'maint_counts', hue = 'comp', data = maint_pd_grp)

# setting axes labels and titles  
axes.set_xlabel('Dates')
axes.set_ylabel('Number of Maitenance Done')

# show plot
plt.show()

In [18]:
# format datetime field which comes in as string
maint_pd['datetime'] = pd.to_datetime(maint_pd['datetime'], format="%Y-%m-%d %H:%M:%S") 

# randomly slicing data with a single machine, machineID = "123"    
plot_data = telemetry_pd.loc[telemetry_pd['machineID'] == 123] 

# further slicing data by date range April through June  
plot_data = maint_pd.loc[(maint_pd['datetime'] >= pd.to_datetime('2015-02-1')) &
                        (maint_pd['datetime'] <= pd.to_datetime('2015-03-1'))]


# seaborn styling 
sns.set_style("whitegrid")

# contoliing plot size 
fig, axes = plt.subplots(1,1, figsize=(12,6))

# grouping failures data   
maint_pd_grp = plot_data.groupby(['datetime', 'comp']).size().reset_index(name = 'maint_counts')

# plotting by diffferent failures
sns.lineplot(x = 'datetime', y = 'maint_counts', hue = 'comp', data = maint_pd_grp)

# setting axes labels and titles  
axes.set_title('Maintenance logs over two-year period')
axes.set_xlabel('Dates')
axes.set_ylabel('Number of Maitenance Done')

# show plot
plt.show()

In [19]:
# matplotlib styling 
plt.style.use('seaborn-whitegrid')
# contoliing plot size 
fig = plt.figure(figsize=(12,4))

# matplotlib histogram
plt.hist(errors_pd['errorID'], color = 'goldenrod', edgecolor = 'goldenrod') 

plt.xlabel('Error Types')
plt.ylabel('Number of Error Incidents')
plt.show()

In [20]:
# format datetime field which comes in as string
errors_pd['datetime'] = pd.to_datetime(errors_pd['datetime'], format="%Y-%m") 


# seaborn styling 
sns.set_style("whitegrid")

# contoliing plot size 
fig, axes = plt.subplots(1,1, figsize=(12,6))
# fig = plt.figure(figsize=(12,4))

# grouping errors data   
errors_pd_grp = errors_pd.groupby(['datetime', 'errorID']).size().reset_index(name = 'error_counts')

# plotting by diffferent errorIDs
sns.lineplot(x = 'datetime', y ='error_counts', hue = 'errorID', data = errors_pd_grp)

# setting axes labels and titles  
axes.set_title('Throughout the 1 -year period, error3 had been the most frequent')
axes.set_xlabel('Dates')
axes.set_ylabel('Number of Error Incidents')
# show plot
plt.show()

In [21]:
# matplotlib styling 
plt.style.use('seaborn-whitegrid')
# contoliing plot size 
fig = plt.figure(figsize=(12,4))

# matplotlib histogram
plt.hist(failures_pd['failure'], color = 'darkred', edgecolor = 'darkred') 

plt.xlabel('Failure Types (by components)')
plt.ylabel('Number of Failure Incidents')
plt.title('Across all the 1000 machines, comp2 had the highest incidence of failures') 
plt.show()

In [22]:
# format datetime field which comes in as string
failures_pd['datetime'] = pd.to_datetime(failures_pd['datetime'], format="%Y-%m-%d %H:%M:%S") 

# seaborn styling 
sns.set_style("whitegrid")

# contoliing plot size 
fig, axes = plt.subplots(1,1, figsize=(12,6))
# fig = plt.figure(figsize=(12,4))

# grouping failures data   
failures_pd_grp = failures_pd.groupby(['datetime', 'failure']).size().reset_index(name = 'failure_counts')

# plotting by diffferent failures
sns.lineplot(x = 'datetime', y ='failure_counts', hue = 'failure', data = failures_pd_grp)

# setting axes labels and titles  
axes.set_title('Throughout the 1 -year period, comp2 had failed the most frequently')
axes.set_xlabel('Dates')
axes.set_ylabel('Number of Failure Incidents')

# show plot
plt.show()

In [23]:
# format datetime field which comes in as string
telemetry_pd['datetime'] = pd.to_datetime(telemetry_pd['datetime'], format="%Y-%m-%d %H:%M:%S") 

# randomly slicing data with a single machine, machineID = "123"    
plot_data = telemetry_pd.loc[telemetry_pd['machineID'] == 123] 

# further slicing data by date range  
plot_data = plot_data.loc[(plot_data['datetime'] >= pd.to_datetime('2015-02-01')) &
                        (plot_data['datetime'] <= pd.to_datetime('2015-03-01'))]

# converting broad dataframe to long datframe using melt
plot_data = pd.melt(plot_data, id_vars=['datetime', 'machineID'])

In [24]:
# seaborn styling 
sns.set_style("whitegrid")



# plotting by diffferent failures
sns.lineplot(x = 'datetime', y ='failure_counts', hue = 'failure', data = failures_pd_grp)

# setting axes labels and titles  
axes.set_title('Throughout the 1 -year period, comp2 had failed the most frequently')
axes.set_xlabel('Dates')
axes.set_ylabel('Number of Failure Incidents')

# show plot
plt.show() 


#### Feature Engineering


In [25]:
## Setup our environment by importing required libraries
import pyspark.sql.functions as F
import pyspark

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, round
from pyspark.sql.functions import datediff
from pyspark.sql.types import DoubleType
from pyspark.sql.window import Window

spark = SparkSession.builder.getOrCreate()

##### Telemetry Features


In [26]:
telemetry = spark.table(TELEMETRY_TABLE).cache()

# rolling mean and standard deviation
# Temporary storage for rolling means
tel_mean = telemetry

# Which features are we interested in telemetry data set
rolling_features = ['volt','rotate', 'pressure', 'vibration']
      
# n hours = n * 3600 seconds  
time_val = 12 * 3600

# Choose the time_val hour timestamps to align the data
# dt_truncated looks at the column named "datetime" in the current data set.
# remember that Spark is lazy... this doesn't execute until it is in a withColumn statement.
dt_truncated = ((round(unix_timestamp(col("datetime")) / time_val) * time_val).cast("timestamp"))

In [27]:
# We choose windows for our rolling windows 12hrs, 24 hrs and 36 hrs
lags = [12, 24, 36]

# align the data
for lag_n in lags:
    wSpec = Window.partitionBy('machineID').orderBy('datetime').rowsBetween(1-lag_n, 0)
    for col_name in rolling_features:
        tel_mean = tel_mean.withColumn(col_name+'_rollingmean_'+str(lag_n), 
                                       F.avg(col(col_name)).over(wSpec))
        tel_mean = tel_mean.withColumn(col_name+'_rollingstd_'+str(lag_n), 
                                       F.stddev(col(col_name)).over(wSpec))

# Calculate lag values...
telemetry_feat = (tel_mean.withColumn("dt_truncated", dt_truncated)
                  .drop('volt', 'rotate', 'pressure', 'vibration')
                  .fillna(0)
                  .groupBy("machineID","dt_truncated")
                  .agg(F.mean('volt_rollingmean_12').alias('volt_rollingmean_12'),
                       F.mean('rotate_rollingmean_12').alias('rotate_rollingmean_12'), 
                       F.mean('pressure_rollingmean_12').alias('pressure_rollingmean_12'), 
                       F.mean('vibration_rollingmean_12').alias('vibration_rollingmean_12'), 
                       F.mean('volt_rollingmean_24').alias('volt_rollingmean_24'),
                       F.mean('rotate_rollingmean_24').alias('rotate_rollingmean_24'), 
                       F.mean('pressure_rollingmean_24').alias('pressure_rollingmean_24'), 
                       F.mean('vibration_rollingmean_24').alias('vibration_rollingmean_24'),
                       F.mean('volt_rollingmean_36').alias('volt_rollingmean_36'),
                       F.mean('vibration_rollingmean_36').alias('vibration_rollingmean_36'),
                       F.mean('rotate_rollingmean_36').alias('rotate_rollingmean_36'), 
                       F.mean('pressure_rollingmean_36').alias('pressure_rollingmean_36'), 
                       F.stddev('volt_rollingstd_12').alias('volt_rollingstd_12'),
                       F.stddev('rotate_rollingstd_12').alias('rotate_rollingstd_12'), 
                       F.stddev('pressure_rollingstd_12').alias('pressure_rollingstd_12'), 
                       F.stddev('vibration_rollingstd_12').alias('vibration_rollingstd_12'), 
                       F.stddev('volt_rollingstd_24').alias('volt_rollingstd_24'),
                       F.stddev('rotate_rollingstd_24').alias('rotate_rollingstd_24'), 
                       F.stddev('pressure_rollingstd_24').alias('pressure_rollingstd_24'), 
                       F.stddev('vibration_rollingstd_24').alias('vibration_rollingstd_24'),
                       F.stddev('volt_rollingstd_36').alias('volt_rollingstd_36'),
                       F.stddev('rotate_rollingstd_36').alias('rotate_rollingstd_36'), 
                       F.stddev('pressure_rollingstd_36').alias('pressure_rollingstd_36'), 
                       F.stddev('vibration_rollingstd_36').alias('vibration_rollingstd_36'), )).cache()

##### Errors features


In [28]:
errors = spark.table(ERRORS_TABLE).cache()

# create a column for each errorID 
error_ind = (errors.groupBy("machineID","datetime","errorID").pivot('errorID')
             .agg(F.count('machineID').alias('dummy')).drop('errorID').fillna(0)
             .groupBy("machineID","datetime")
             .agg(F.sum('error1').alias('error1sum'), 
                  F.sum('error2').alias('error2sum'), 
                  F.sum('error3').alias('error3sum'), 
                  F.sum('error4').alias('error4sum'), 
                  F.sum('error5').alias('error5sum')))

# join the telemetry data with errors
error_count = (telemetry.join(error_ind, 
                              ((telemetry['machineID'] == error_ind['machineID']) 
                               & (telemetry['datetime'] == error_ind['datetime'])), "left")
               .drop('volt', 'rotate', 'pressure', 'vibration')
               .drop(error_ind.machineID).drop(error_ind.datetime)
               .fillna(0))

error_features = ['error1sum','error2sum', 'error3sum', 'error4sum', 'error5sum']

wSpec = Window.partitionBy('machineID').orderBy('datetime').rowsBetween(1-24, 0)
for col_name in error_features:
    # We're only interested in the erros in the previous 24 hours.
    error_count = error_count.withColumn(col_name+'_rollingmean_24', 
                                         F.avg(col(col_name)).over(wSpec))

error_feat = (error_count.withColumn("dt_truncated", dt_truncated)
              .drop('error1sum', 'error2sum', 'error3sum', 'error4sum', 'error5sum').fillna(0)
              .groupBy("machineID","dt_truncated")
              .agg(F.mean('error1sum_rollingmean_24').alias('error1sum_rollingmean_24'), 
                   F.mean('error2sum_rollingmean_24').alias('error2sum_rollingmean_24'), 
                   F.mean('error3sum_rollingmean_24').alias('error3sum_rollingmean_24'), 
                   F.mean('error4sum_rollingmean_24').alias('error4sum_rollingmean_24'), 
                   F.mean('error5sum_rollingmean_24').alias('error5sum_rollingmean_24')))

##### Maintenance Feature Engineering

In [29]:
maint = spark.table(MAINT_TABLE).cache()

# create a column for each component replacement
maint_replace = (maint.groupBy("machineID","datetime","comp").pivot('comp')
                 .agg(F.count('machineID').alias('dummy')).fillna(0)
                 .groupBy("machineID","datetime")
                 .agg(F.sum('comp1').alias('comp1sum'), 
                      F.sum('comp2').alias('comp2sum'), 
                      F.sum('comp3').alias('comp3sum'),
                      F.sum('comp4').alias('comp4sum'))
                 .withColumnRenamed('datetime','datetime_maint')).cache()

In [30]:
# We want to align the component information on telemetry features timestamps.
telemetry_times = (telemetry_feat.select(telemetry_feat.machineID, telemetry_feat.dt_truncated)
                   .withColumnRenamed('dt_truncated','datetime_tel')).cache()

In [31]:
def grab_component_records(self, telemetry_times, comp_sum="comp1sum", sincelastcomp_="sincelastcomp1",
                          comp_a='comp2sum', comp_b='comp3sum', comp_c='comp4sum'):
    maint_comp = (self.where(col(comp_sum) == '1').withColumnRenamed('datetime', 'datetime_maint')
                  .drop(comp_a, comp_b, comp_c)).cache()
    # Within each machine, get the last replacement date for each timepoint
    maint_tel_comp = (telemetry_times.join(maint_comp,
                                           ((telemetry_times['machineID'] == maint_comp['machineID'])
                                            & (telemetry_times['datetime_tel'] > maint_comp['datetime_maint'])
                                            & (maint_comp[(comp_sum)] == '1')))
                      .drop(maint_comp.machineID)).cache()
    # Calculate the number of days between replacements
    return (maint_tel_comp.withColumn(sincelastcomp_,
                                      datediff(maint_tel_comp.datetime_tel, maint_tel_comp.datetime_maint))
            .drop(maint_tel_comp.datetime_maint).drop(maint_tel_comp[comp_sum])).cache()

pyspark.sql.dataframe.DataFrame.grab_component_records = grab_component_records

# Grab component 1 records
comp1 = maint_replace.grab_component_records(telemetry_times, comp_sum="comp1sum", sincelastcomp_="sincelastcomp1",
                                            comp_a='comp2sum', comp_b='comp3sum', comp_c='comp4sum').cache()
comp2 = maint_replace.grab_component_records(telemetry_times, comp_sum="comp2sum", sincelastcomp_="sincelastcomp2",
                                            comp_a='comp1sum', comp_b='comp3sum', comp_c='comp4sum').cache()
comp3 = maint_replace.grab_component_records(telemetry_times, comp_sum="comp3sum", sincelastcomp_="sincelastcomp3",
                                            comp_a='comp1sum', comp_b='comp2sum', comp_c='comp4sum').cache()
comp4 = maint_replace.grab_component_records(telemetry_times, comp_sum="comp4sum", sincelastcomp_="sincelastcomp4",
                                            comp_a='comp1sum', comp_b='comp2sum', comp_c='comp3sum').cache()

In [32]:
comp3_4 = (comp3.join(comp4, ((comp3['machineID'] == comp4['machineID']) 
                              & (comp3['datetime_tel'] == comp4['datetime_tel'])), "left")
           .drop(comp4.machineID).drop(comp4.datetime_tel)).cache()

# Join component 2 to 3 and 4
comp2_3_4 = (comp2.join(comp3_4, ((comp2['machineID'] == comp3_4['machineID']) 
                                  & (comp2['datetime_tel'] == comp3_4['datetime_tel'])), "left")
             .drop(comp3_4.machineID).drop(comp3_4.datetime_tel)).cache()

# Join component 1 to 2, 3 and 4
comps_feat = (comp1.join(comp2_3_4, ((comp1['machineID'] == comp2_3_4['machineID']) 
                                      & (comp1['datetime_tel'] == comp2_3_4['datetime_tel'])), "left")
               .drop(comp2_3_4.machineID).drop(comp2_3_4.datetime_tel)
               .groupBy("machineID", "datetime_tel")
               .agg(F.max('sincelastcomp1').alias('sincelastcomp1'), 
                    F.max('sincelastcomp2').alias('sincelastcomp2'), 
                    F.max('sincelastcomp3').alias('sincelastcomp3'), 
                    F.max('sincelastcomp4').alias('sincelastcomp4'))
               .fillna(0)).cache()

# Choose the time_val hour timestamps to align the data
dt_truncated = ((round(unix_timestamp(col("datetime_tel")) / time_val) * time_val).cast("timestamp"))

# Collect data
maint_feat = (comps_feat.withColumn("dt_truncated", dt_truncated)
              .groupBy("machineID","dt_truncated")
              .agg(F.mean('sincelastcomp1').alias('comp1sum'), 
                   F.mean('sincelastcomp2').alias('comp2sum'), 
                   F.mean('sincelastcomp3').alias('comp3sum'), 
                   F.mean('sincelastcomp4').alias('comp4sum'))).cache()

##### Machine Features


In [33]:
machines = spark.table(MACHINES_TABLE).cache()

# one hot encoding of the variable model, basically creates a set of dummy boolean variables
catVarNames = ['model']  
sIndexers = [StringIndexer(inputCol=x, outputCol=x + '_indexed') for x in catVarNames]
machines_cat = Pipeline(stages=sIndexers).fit(machines).transform(machines)

# one-hot encode
ohEncoders = [OneHotEncoder(inputCol=x + '_indexed', outputCol=x + '_encoded')
              for x in catVarNames]

ohPipelineModel = Pipeline(stages=ohEncoders).fit(machines_cat)
machines_cat = ohPipelineModel.transform(machines_cat)

drop_list = [col_n for col_n in machines_cat.columns if 'indexed' in col_n]

machines_feat = machines_cat.select([column for column in machines_cat.columns if column not in drop_list]).cache()


##### Combining Features


In [34]:
# join error features with component maintenance features
error_maint = (error_feat.join(maint_feat, 
                               ((error_feat['machineID'] == maint_feat['machineID']) 
                                & (error_feat['dt_truncated'] == maint_feat['dt_truncated'])), "left")
               .drop(maint_feat.machineID).drop(maint_feat.dt_truncated))

# now join that with machines features
error_maint_feat = (error_maint.join(machines_feat, 
                                     ((error_maint['machineID'] == machines_feat['machineID'])), "left")
                    .drop(machines_feat.machineID))

# Clean up some unecessary columns
error_maint_feat = error_maint_feat.select([c for c in error_maint_feat.columns if c not in 
                                            {'error1sum', 'error2sum', 'error3sum', 'error4sum', 'error5sum'}])

# join telemetry with error/maint/machine features to create final feature matrix
final_feat = (telemetry_feat.join(error_maint_feat, 
                                  ((telemetry_feat['machineID'] == error_maint_feat['machineID']) 
                                   & (telemetry_feat['dt_truncated'] == error_maint_feat['dt_truncated'])), "left")
              .drop(error_maint_feat.machineID).drop(error_maint_feat.dt_truncated))

##### Label Construction


In [35]:
failures = spark.table(FAILURES_TABLE).cache()

# We need to redefine dt_truncated to align with the failures table
dt_truncated = ((round(unix_timestamp(col("datetime")) / time_val) * time_val).cast("timestamp"))

fail_diff = (failures.withColumn("dt_truncated", dt_truncated)
             .drop(failures.datetime))

In [36]:
# map the failure data to final feature matrix
labeled_features = (final_feat.join(fail_diff, 
                                    ((final_feat['machineID'] == fail_diff['machineID']) 
                                     & (final_feat['dt_truncated'] == fail_diff['dt_truncated'])), "left")
                    .drop(fail_diff.machineID).drop(fail_diff.dt_truncated)
                    .withColumn('failure', F.when(col('failure') == "comp1", 1.0).otherwise(col('failure')))
                    .withColumn('failure', F.when(col('failure') == "comp2", 2.0).otherwise(col('failure')))
                    .withColumn('failure', F.when(col('failure') == "comp3", 3.0).otherwise(col('failure')))
                    .withColumn('failure', F.when(col('failure') == "comp4", 4.0).otherwise(col('failure'))))

labeled_features = (labeled_features.withColumn("failure", 
                                                labeled_features.failure.cast(DoubleType()))
                    .fillna(0))

In [37]:
# lag values to manually backfill label (bfill =7)
my_window = Window.partitionBy('machineID').orderBy(labeled_features.dt_truncated.desc())

# Create the previous 7 days 
labeled_features = (labeled_features.withColumn("prev_value1", 
                                                F.lag(labeled_features.failure).
                                                over(my_window)).fillna(0))
labeled_features = (labeled_features.withColumn("prev_value2", 
                                                F.lag(labeled_features.prev_value1).
                                                over(my_window)).fillna(0))
labeled_features = (labeled_features.withColumn("prev_value3", 
                                                F.lag(labeled_features.prev_value2).
                                                over(my_window)).fillna(0))
labeled_features = (labeled_features.withColumn("prev_value4", 
                                                F.lag(labeled_features.prev_value3).
                                                over(my_window)).fillna(0)) 
labeled_features = (labeled_features.withColumn("prev_value5", 
                                                F.lag(labeled_features.prev_value4).
                                                over(my_window)).fillna(0)) 
labeled_features = (labeled_features.withColumn("prev_value6", 
                                                F.lag(labeled_features.prev_value5).
                                                over(my_window)).fillna(0))
labeled_features = (labeled_features.withColumn("prev_value7", 
                                                F.lag(labeled_features.prev_value6).
                                                over(my_window)).fillna(0))

# Create a label features
labeled_features = (labeled_features.withColumn('label', labeled_features.failure + 
                                                labeled_features.prev_value1 +
                                                labeled_features.prev_value2 +
                                                labeled_features.prev_value3 +
                                                labeled_features.prev_value4 +
                                                labeled_features.prev_value5 + 
                                                labeled_features.prev_value6 + 
                                                labeled_features.prev_value7))

# Restrict the label to be on the range of 0:4, and remove extra columns
labeled_features = (labeled_features.withColumn('label_e', F.when(col('label') > 4, 4.0)
                                                .otherwise(col('label')))
                    .drop(labeled_features.prev_value1).drop(labeled_features.prev_value2)
                    .drop(labeled_features.prev_value3).drop(labeled_features.prev_value4)
                    .drop(labeled_features.prev_value5).drop(labeled_features.prev_value6)
                    .drop(labeled_features.prev_value7).drop(labeled_features.label))

if not(spark.catalog._jcatalog.tableExists(FEATURES_TABLE)):
   labeled_features.write.saveAsTable(FEATURES_TABLE)

In [7]:
final_df = spark.table(FEATURES_TABLE).toPandas()

/opt/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:2103: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.

In [ ]:
## setting up the credentials for ADLS Gen2
import os
from azure.storage.blob import BlockBlobService

# setting up blob storage configs
STORAGE_ACCOUNT_NAME = "#DATA_LAKE_NAME#"
STORAGE_ACCOUNT_ACCESS_KEY = "#STORAGE_ACCOUNT_KEY#"
STORAGE_CONTAINER_NAME = "azureml-mfg"

blob_service = BlockBlobService(STORAGE_ACCOUNT_NAME, STORAGE_ACCOUNT_ACCESS_KEY) 

# Saving the final dataframe to csv 
final_df.to_csv('mfg_pdm.csv',index=False)


output_file_path=os.path.join(os.getcwd(), "mfg_pdm.csv")
output_blob_file= "mfg_pdm.csv"

# uploading the csv to  the ADLSGen2 storage container
blob_service.create_blob_from_path(STORAGE_CONTAINER_NAME,output_blob_file ,output_file_path)